***
# `Project:` Prevendo a <font color='blue'>morte</font> ou <font color='blue'>vida</font> de pacientes com hepatite

## `Date:` fevereiro, 2022

## `Data Scientist:` Walter Trevisan
***

<a name='notebook-header'></a>
## `Deploy do Modelo Preditivo`

Nesta etapa, vamos utilizar o **modelo preditivo escolhido** para prever se **novos pacientes** (com hepatite) irão *morrer* ou *viver* de acordo com algumas informações clínicas sobre cada paciente.

### Conteúdo:

1. [Setup Inicial](#initial-setup);

2. [Carregar os dados clínicos dos **novos pacientes**](#load-data);

3. [Preparar os dados dos **novos pacientes**](#data-munging);

4. [Prever se os novos pacientes irão **morrer** ou **viver**](#predict);

5. [Conclusão final](#conclusion).

___
<a name='initial-setup'></a>
## <font color='blue'>1- Setup Inicial:</font>

Primeiro, vamos carregar os **pacotes e funções** que serão utilizadas neste **notebook**.

In [1]:
# As novas versões do Pandas e Matplotlib trazem diversas mensagens de aviso ao desenvolvedor.
# Então, vamos desativar essas mensagens.
import sys # O pacote "sys" permite manipulações com o sistema operacional:
import os  # Operation System (Packages and Functions)
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Importa função para verificarmos a versão da linguagem python:
from platform import python_version

# Importando os pacote NumPy:
import numpy as np
# Importando os pacote Pandas:
import pandas as pd

# Definindo o diretório raiz (Root) onde serão armazenados todas as informações
# (Imagens, Gráficos, Objetos, Dados, Modelos de ML, etc...) do projeto.
# Diretório Raiz (Root) do Projeto:
ROOT_DIR = "."

# Path: onde ficarão armazenados os "Objetos" (Estrututras de Dados) relacionados ao Projeto:
OBJ_PATH = os.path.join(ROOT_DIR, "objects")
# Criando o diretório, se ele não existir:
os.makedirs(OBJ_PATH, exist_ok=True)

# Path: onde ficarão armazenados os "datasets" (arquivos "csv") e os "objetos" (Data Frames) do Projeto:
DATA_PATH = os.path.join(ROOT_DIR, "data")
# Path do arquivo "data" que contém os dados dos novos pacientes:
DATA_FILE_PATH = os.path.join(DATA_PATH, "new_patients.data")
# Criando o diretório, se ele não existir:
os.makedirs(DATA_PATH, exist_ok=True)

# Path: onde ficarão armazenados os "resultados" (arquivos "csv") das previsões dos "novos pacientes":
DATA_PATH = os.path.join(ROOT_DIR, "outcome")
# Path do arquivo "csv" que contém as previsões dos novos pacientes:
OUTCOME_FILE_PATH = os.path.join(DATA_PATH, "results.csv")
# Criando o diretório, se ele não existir:
os.makedirs(DATA_PATH, exist_ok=True)

# Path: onde serão armazenadas as "Imagens" (Figuras e Gráficos) do Projeto:
GRAPHICS_PATH = os.path.join(ROOT_DIR, "images", "graphics")
# Criando o diretório, se ele não existir:
os.makedirs(GRAPHICS_PATH, exist_ok=True)

# Path: onde ficarão armazenados os "Modelos Preditivos" (Machine Learning) relacionados ao Projeto:
ML_PATH = os.path.join(ROOT_DIR, "models")
# Criando o diretório, se ele não existir:
os.makedirs(ML_PATH, exist_ok=True)

# Path: onde estão armazenadas as classes e funções que serão utilizadas neste notebook:
LIB_PATH = os.path.join(ROOT_DIR, "library")

# Adicionando o diretório "./library" ao 'path' do Sistema, para podermos importar classes e funções que serão
# utilizadas neste notebook:
sys.path.append(LIB_PATH)

# Importando para este notebook, as classes e funções definidas no módulo "data_science_library.py":
import data_science_library as dslib

# Importando para este notebook, as classes e funções definidas no módulo "project_library.py":
import project_library as pjlib

print("Setup Complete!")

Setup Complete!


In [2]:
# Versões dos pacotes usados neste jupyter notebook:
print("Versões dos pacotes usados neste jupyter notebook:")
print("Python      : {}".format(python_version()))
print("Numpy       : {}".format(np.__version__))
print("Pandas      : {}".format(pd.__version__))

Versões dos pacotes usados neste jupyter notebook:
Python      : 3.8.12
Numpy       : 1.19.5
Pandas      : 1.3.5


___
<a name='load-data'></a>
## <font color='blue'>2- Carregar os dados clínicos dos `novos pacientes`:</font>

In [3]:
# Definindo os nomes das variáveis (features):
features = ['Patient','Age','Gender','Steroid','Antivirals','Fatigue','Malaise','Anorexia','LiverBig',
            'LiverFirm','SpleenPalpable','Spiders','Ascites','Varices','Bilirubin','AlkPhosphate','SGOT',
            'Albumin','Protime','Histology']

# Lendo os dados e criando um data frame:
new_patients_df = pd.read_csv(
    filepath_or_buffer=DATA_FILE_PATH,
    names = features,
    sep = ',',
    encoding = 'latin-1'
)

# Visualizando os primeiros registros:
new_patients_df.head(10)

,Patient,Age,Gender,Steroid,Antivirals,Fatigue,Malaise,Anorexia,LiverBig,LiverFirm,SpleenPalpable,Spiders,Ascites,Varices,Bilirubin,AlkPhosphate,SGOT,Albumin,Protime,Histology
0,p1,30,1,2,2,2,2,2,2,2,2,2,2,2,1.0,?,120,3.9,?,1
1,p2,41,2,2,1,1,1,1,2,2,2,2,2,2,0.7,81,53,5.0,74,1
2,p3,58,1,2,2,1,2,2,1,1,1,1,2,2,2.0,167,242,3.3,?,1
3,p4,56,1,1,2,1,1,1,1,1,2,1,2,2,2.9,90,153,4.0,?,2
4,p5,48,1,2,2,1,1,1,2,1,2,1,2,2,2.0,158,278,3.8,?,2
5,p6,43,1,2,2,1,2,2,2,2,1,1,1,2,1.2,100,19,3.1,42,2


Os dados acima representam as **informações clínicas** de **seis novos pacientes** que contrairam a *hepatite*.

___
<a name='data-munging'></a>
## <font color='blue'>3- Preparar os dados dos `novos pacientes`:</font>

### Carregando *objetos* para tratamento de valores ausentes:

In [4]:
# Carregando o objeto "imputer_mv" do processo de "imputação multivariada" para tratamento dos
# valores ausentes das variáveis numéricas:
imputer_mv = dslib.pickle_object_load(path=OBJ_PATH, file="imputer_mv.pkl", msg=None)

# Carregando o objeto "imputer_cat" do processo de "imputação univariada" das variáveis categóricas:
imputer_cat = dslib.pickle_object_load(path=OBJ_PATH, file="imputer_cat.pkl", msg=None)

### Carregando *objeto* com as informações do modelo `KNN`:

In [5]:
# Carregando as informações do melhor modelo preditivo criado com o "KNN":
knn = dslib.pickle_object_load (path=ML_PATH, file="knn_best_model.pkl", msg=None)
knn

{'variables': ['Gender',
  'Steroid',
  'Antivirals',
  'Fatigue',
  'Malaise',
  'Anorexia',
  'LiverBig',
  'LiverFirm',
  'SpleenPalpable',
  'Spiders',
  'Ascites',
  'Varices',
  'Histology',
  'Age',
  'Bilirubin',
  'AlkPhosphate',
  'SGOT',
  'Albumin',
  'Protime'],
 'scaler': StandardScaler(),
 'model': KNeighborsClassifier(metric='manhattan', n_jobs=-1, n_neighbors=15, p=1,
                      weights='distance')}

### Preparando as variáveis categóricas e numéricas de acordo com o contexto:

In [6]:
# Separando o "Id" dos pacientes para retorno dos resultados:
patient_id = new_patients_df[['Patient']]

# Separando os dados clínicos dos pacientes:
data_df = new_patients_df.iloc[:, 1:]
data_df

,Age,Gender,Steroid,Antivirals,Fatigue,Malaise,Anorexia,LiverBig,LiverFirm,SpleenPalpable,Spiders,Ascites,Varices,Bilirubin,AlkPhosphate,SGOT,Albumin,Protime,Histology
0,30,1,2,2,2,2,2,2,2,2,2,2,2,1.0,?,120,3.9,?,1
1,41,2,2,1,1,1,1,2,2,2,2,2,2,0.7,81,53,5.0,74,1
2,58,1,2,2,1,2,2,1,1,1,1,2,2,2.0,167,242,3.3,?,1
3,56,1,1,2,1,1,1,1,1,2,1,2,2,2.9,90,153,4.0,?,2
4,48,1,2,2,1,1,1,2,1,2,1,2,2,2.0,158,278,3.8,?,2
5,43,1,2,2,1,2,2,2,2,1,1,1,2,1.2,100,19,3.1,42,2


In [7]:
# Preparando as variáveis categóricas e numéricas dos dados:
data_df = pjlib.data_prep_step_1(df=data_df, target=False)
# Cria uma cópia para retorno dos resultados:
patient_data = data_df.copy(deep=True)
# Mostra os dados clínicos:
patient_data

,Age,Gender,Steroid,Antivirals,Fatigue,Malaise,Anorexia,LiverBig,LiverFirm,SpleenPalpable,Spiders,Ascites,Varices,Bilirubin,AlkPhosphate,SGOT,Albumin,Protime,Histology
0,30.0,male,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,10.0,NaN,120.0,39.0,NaN,no
1,41.0,female,yes,no,no,no,no,yes,yes,yes,yes,yes,yes,7.0,81.0,53.0,50.0,74.0,no
2,58.0,male,yes,yes,no,yes,yes,no,no,no,no,yes,yes,20.0,167.0,242.0,33.0,NaN,no
3,56.0,male,no,yes,no,no,no,no,no,yes,no,yes,yes,29.0,90.0,153.0,40.0,NaN,yes
4,48.0,male,yes,yes,no,no,no,yes,no,yes,no,yes,yes,20.0,158.0,278.0,38.0,NaN,yes
5,43.0,male,yes,yes,no,yes,yes,yes,yes,no,no,no,yes,12.0,100.0,19.0,31.0,42.0,yes


### Definindo as variáveis categóricas e numéricas:

In [8]:
# Definindo as variáveis categóricas preditoras:
cat_variables = ['Gender', 'Steroid', 'Antivirals', 'Fatigue', 'Malaise', 'Anorexia', 'LiverBig', 'LiverFirm',
                 'SpleenPalpable', 'Spiders', 'Ascites', 'Varices', 'Histology']
# Definindo as variáveis numéricas:
num_variables = ['Age', 'Bilirubin', 'AlkPhosphate', 'SGOT', 'Albumin', 'Protime']

### Tratando os valores ausentes das variáveis categóricas:

In [9]:
# Separando as variáveis "categóricas" preditoras em um outro dataframe:
data_cat_df = data_df[cat_variables].copy(deep=True)
# Substituindo os valores ausentes:
data_cat_tf = imputer_cat.transform(data_cat_df.values)
# Remontando o dataframe original sem os valores ausentes:
data_cat_df = pd.DataFrame(data_cat_tf, columns=cat_variables)
# Verificando se os "missing values" foram removidos:
df_mv = dslib.missing_values(data=data_cat_df, axis=0, threshold=0.01, target=None)
print('Quantidade de variáveis: {}'.format(len(df_mv)))
df_mv

Quantidade de variáveis: 0


Attributes (Columns),Total,Percent


### Aplicando o método *`encoding`* nas variáveis `categóricas`:

In [10]:
# Aplicando o método "encoding" nas variáveis categóricas:
data_cat_df = pjlib.method_encoding_categorical(data_cat_df, target=False)
# Mostra os resultados:
data_cat_df

,Gender,Steroid,Antivirals,Fatigue,Malaise,Anorexia,LiverBig,LiverFirm,SpleenPalpable,Spiders,Ascites,Varices,Histology
0,0,1,1,1,1,1,1,1,1,1,1,1,0
1,1,1,0,0,0,0,1,1,1,1,1,1,0
2,0,1,1,0,1,1,0,0,0,0,1,1,0
3,0,0,1,0,0,0,0,0,1,0,1,1,1
4,0,1,1,0,0,0,1,0,1,0,1,1,1
5,0,1,1,0,1,1,1,1,0,0,0,1,1


### Tratando os valores ausentes das variáveis numéricas:

In [11]:
# Criando um dataframe apenas com as variáveis numéricas:
data_num_df = data_df[num_variables].copy(deep=True)
# Verificando variáveis com valores iguais a "0":
# Criando um índice de todos os valores iguais a "0":
index = data_num_df.values == 0
data_num_df.loc[index,:]

,Age,Bilirubin,AlkPhosphate,SGOT,Albumin,Protime


In [12]:
# Verificando as variáveis numéricas com "missing values":
df_mv = dslib.missing_values(data=data_num_df, axis=0, threshold=0.01, target=None)
print('Quantidade de variáveis: {}'.format(len(df_mv)))
df_mv

Quantidade de variáveis: 2


Attributes (Columns),Total,Percent
Protime,4,0.666667
AlkPhosphate,1,0.166667


In [13]:
# Preparando um dataframe para aplicarmos a "imputação múltipla":
var_num_df = pd.concat([data_cat_df[cat_variables], data_num_df], axis=1)
# Verificando o shape:
var_num_df.shape

(6, 19)

In [14]:
# Aplicando a imputação múltipla:
data_imp_tf = imputer_mv.transform(var_num_df)

In [15]:
# Salva os valores ausentes tratados no mesmo dataframe:
var_num_df = pd.DataFrame(data_imp_tf, columns = var_num_df.columns)

# Refazendo o dataframe com todos os valores ausentes tratados nas variáveis numéricas:
data_num_df = var_num_df[num_variables].copy(deep=True)

# Verificando se todos os valores ausentes foram tratados:
df_mv = dslib.missing_values(data=data_num_df, target=None)
print('Quantidade de variáveis: {}'.format(len(df_mv)))
df_mv

Quantidade de variáveis: 6


Attributes (Columns),Total,Percent
Age,0,0.0
Bilirubin,0,0.0
AlkPhosphate,0,0.0
SGOT,0,0.0
Albumin,0,0.0
Protime,0,0.0


### Aplicando a padronização de escala nas variáveis numéricas:

In [16]:
# Padronização de escala:
var_num_tf = knn['scaler'].transform(X=data_num_df.values)
# Recriando o data frame com as variáveis padronizadas:
data_num_df = pd.DataFrame(var_num_tf, columns = data_num_df.columns)
# Mostra os dados padronizados:
data_num_df

,Age,Bilirubin,AlkPhosphate,SGOT,Albumin,Protime
0,-0.892559,-0.335005,-0.310904,0.473101,0.088050,1.061513
1,-0.021251,-0.579201,-0.550565,-0.366244,1.771178,0.450963
2,1.325314,0.478982,1.177080,2.001461,-0.830020,-0.920123
3,1.166895,1.211570,-0.369765,0.886510,0.241062,0.075759
4,0.533217,0.478982,0.996280,2.452453,-0.064962,-0.051077
5,0.137168,-0.172208,-0.168876,-0.792181,-1.136043,-1.092292


### Criando o *dataframe* final com os dados das variáveis preditoras preparados:

In [17]:
# Criando o dataframe final:
X = pd.concat([data_cat_df, data_num_df], axis=1)
# Mostrando os dados preparados:
X

,Gender,Steroid,Antivirals,Fatigue,Malaise,Anorexia,LiverBig,LiverFirm,SpleenPalpable,Spiders,Ascites,Varices,Histology,Age,Bilirubin,AlkPhosphate,SGOT,Albumin,Protime
0,0,1,1,1,1,1,1,1,1,1,1,1,0,-0.892559,-0.335005,-0.310904,0.473101,0.088050,1.061513
1,1,1,0,0,0,0,1,1,1,1,1,1,0,-0.021251,-0.579201,-0.550565,-0.366244,1.771178,0.450963
2,0,1,1,0,1,1,0,0,0,0,1,1,0,1.325314,0.478982,1.177080,2.001461,-0.830020,-0.920123
3,0,0,1,0,0,0,0,0,1,0,1,1,1,1.166895,1.211570,-0.369765,0.886510,0.241062,0.075759
4,0,1,1,0,0,0,1,0,1,0,1,1,1,0.533217,0.478982,0.996280,2.452453,-0.064962,-0.051077
5,0,1,1,0,1,1,1,1,0,0,0,1,1,0.137168,-0.172208,-0.168876,-0.792181,-1.136043,-1.092292


___
<a name='data-munging'></a>
## <font color='blue'>4- Prever se os novos pacientes irão `morrer` ou `viver`:</font>

Para fazer a previsão vamos utilizar o **melhor modelo preditivo** que foi escolhido na etapa de *machine learning*.

In [18]:
# Fazendo as previsões:
probs = np.round(knn['model'].predict_proba(X), decimals=1)
result = knn['model'].predict(X)

In [19]:
# Print dos resultados:
for i, p in enumerate(patient_id['Patient'].to_list()):
    print(f"Paciente: {p}")
    print(f"Probabilidade do paciente não sobreviver: {probs[i,0]*100}%")
    print(f"Probabilidade do paciente sobreviver: {probs[i,1]*100}%")
    # Analisando a categoria prevista:
    if(result[i] == 0):
        diagnosis = 'DIE'
    else:
        diagnosis = 'LIVE'
    print(f"Categoria prevista pelo modelo:{diagnosis}")
    print()

Paciente: p1
Probabilidade do paciente não sobreviver: 0.0%
Probabilidade do paciente sobreviver: 100.0%
Categoria prevista pelo modelo:LIVE

Paciente: p2
Probabilidade do paciente não sobreviver: 0.0%
Probabilidade do paciente sobreviver: 100.0%
Categoria prevista pelo modelo:LIVE

Paciente: p3
Probabilidade do paciente não sobreviver: 20.0%
Probabilidade do paciente sobreviver: 80.0%
Categoria prevista pelo modelo:LIVE

Paciente: p4
Probabilidade do paciente não sobreviver: 20.0%
Probabilidade do paciente sobreviver: 80.0%
Categoria prevista pelo modelo:LIVE

Paciente: p5
Probabilidade do paciente não sobreviver: 0.0%
Probabilidade do paciente sobreviver: 100.0%
Categoria prevista pelo modelo:LIVE

Paciente: p6
Probabilidade do paciente não sobreviver: 100.0%
Probabilidade do paciente sobreviver: 0.0%
Categoria prevista pelo modelo:DIE



### Salvando um arquivo `csv` com os resultados das previsões:

In [20]:
# Incluindo o "ID" de cada paciente:
patient_data = patient_id.join(other=patient_data)

# Categorias previstas: 0=DIE e 1=LIVE:
cats = ['DIE', 'LIVE']

# Incluindo as informações no data frame:
patient_data['Predict'] = [cats[i] for i in result]
patient_data['Probability'] = [probs[i, result[i]] for i in range(len(patient_id))]

# Mostrando os resultados preditos pelo modelo:
patient_data

,Patient,Age,Gender,Steroid,Antivirals,Fatigue,Malaise,Anorexia,LiverBig,LiverFirm,...,Ascites,Varices,Bilirubin,AlkPhosphate,SGOT,Albumin,Protime,Histology,Predict,Probability
0,p1,30.0,male,yes,yes,yes,yes,yes,yes,yes,...,yes,yes,10.0,NaN,120.0,39.0,NaN,no,LIVE,1.0
1,p2,41.0,female,yes,no,no,no,no,yes,yes,...,yes,yes,7.0,81.0,53.0,50.0,74.0,no,LIVE,1.0
2,p3,58.0,male,yes,yes,no,yes,yes,no,no,...,yes,yes,20.0,167.0,242.0,33.0,NaN,no,LIVE,0.8
3,p4,56.0,male,no,yes,no,no,no,no,no,...,yes,yes,29.0,90.0,153.0,40.0,NaN,yes,LIVE,0.8
4,p5,48.0,male,yes,yes,no,no,no,yes,no,...,yes,yes,20.0,158.0,278.0,38.0,NaN,yes,LIVE,1.0
5,p6,43.0,male,yes,yes,no,yes,yes,yes,yes,...,no,yes,12.0,100.0,19.0,31.0,42.0,yes,DIE,1.0


In [21]:
# Salvando um arquivo "csv":
patient_data.to_csv(path_or_buf=OUTCOME_FILE_PATH, index=False)

___
<a name='conclusion'></a>
## <font color='blue'>5- Conclusão final:</font>

Portanto, o **modelo preditivo** atende de forma **satisfatória** o problema de negócio para o qual ele foi criado, ou seja, se o modelo preditivo for colocado em produção para prever se um paciente, com hepatite, irá sobreviver (ou não), poderemos esperar que ele tenha um **bom desempenho**.

## <font color='black'>FIM</font>